In [43]:
from bs4 import BeautifulSoup
import json
import pandas as pd
import requests

In [44]:
def convert_html_to_df(soup):
    result = soup.find('script').text
    substring = 'gon.search='
    start = result.find(substring) + len(substring)
    blob = result[start:-8]
    result = json.loads(blob)['schools']
    
    schools = []
    grades = []
    districts = []
    ratings = []
    ratings_test_scores = []
    ratings_student_progress = []
    ratings_equity = []
    students = []
    ratios = []
    
    for res in result:
        schools.append(res.get('name'))
        grades.append(res.get('gradeLevels'))
        districts.append(res.get('districtName'))
        
        ratings.append(res.get('rating', -1))
        sub_rating = res.get('subratings', {})
        ratings_test_scores.append(sub_rating.get('Test Scores Rating', -1))
        ratings_student_progress.append(sub_rating.get('Student Progress Rating', -1))
        ratings_equity.append(sub_rating.get('Equity Overview Rating', -1))
        
        students.append(res.get('enrollment'))
        ratios.append(res.get('studentsPerTeacher'))
    
    return pd.DataFrame({
        'school': schools,
        'district': districts,
        'rating': ratings,
        'rating_scores': ratings_test_scores,
        'rating_progress': ratings_student_progress,
        'rating_equity': ratings_equity,
        'grade': grades,
        'student': students,
        'ratio': ratios
    })

In [45]:
def convert_url_to_df(url):
    hdr = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
    }
    response = requests.get(url, headers=hdr)
    if response.status_code != 200:
        print(response.status_code)
        return
    
    soup = BeautifulSoup(response.text, 'html.parser')
    return convert_html_to_df(soup)

In [40]:
url = 'https://www.greatschools.org/new-jersey/schools/?gradeLevels%5B%5D=e&sort=rating&st%5B%5D=public_charter&st%5B%5D=public&tableView=Academic&view=table'
df = convert_url_to_df(url)

In [42]:
df.head()

,school,district,rating,rating_scores,rating_progress,rating_equity,grade,student,ratio
0,Walter M. Schirra Elementary School,Old Bridge Township School District,10,10,9,10,K-5 & Ungraded,278,10
1,Clark Mills Elementary School,Manalapan-Englishtown Regional School District,10,9,10,10,K-5,508,9
2,Frank Defino Central Elementary School,Marlboro Township Board Of Education School Di...,10,10,10,-1,K-5 & Ungraded,515,11
3,Mendham Twp Middle School,Mendham Township Board Of Education School Dis...,10,10,10,-1,5-8,313,9
4,Edgar Middle School,Metuchen Public School District,10,9,10,10,5-8 & Ungraded,714,12


In [53]:
url_base = (
    'https://www.greatschools.org/new-jersey/schools/?gradeLevels%5B%5D=e&page={}'
    '&sort=rating&st%5B%5D=public_charter&st%5B%5D=public&tableView=Academic&view=table'
)
urls = [ url_base.format(i+1) for i in range(66)]
dfs = [ convert_url_to_df(url) for url in urls ]

In [56]:
df_all = pd.concat(dfs, axis=0)
df_all = df_all.reset_index()
df_all = df_all.rename(columns={df_all.columns[0]: 'ranking'})
df_all = df_all[df_all.rating_scores != -1]

In [57]:
df_all

,ranking,school,district,rating,rating_scores,rating_progress,rating_equity,grade,student,ratio
0,0,Walter M. Schirra Elementary School,Old Bridge Township School District,10,10,9,10,K-5 & Ungraded,278,10.0
1,1,Clark Mills Elementary School,Manalapan-Englishtown Regional School District,10,9,10,10,K-5,508,9.0
2,2,Frank Defino Central Elementary School,Marlboro Township Board Of Education School Di...,10,10,10,-1,K-5 & Ungraded,515,11.0
3,3,Mendham Twp Middle School,Mendham Township Board Of Education School Dis...,10,10,10,-1,5-8,313,9.0
4,4,Edgar Middle School,Metuchen Public School District,10,9,10,10,5-8 & Ungraded,714,12.0
...,...,...,...,...,...,...,...,...,...,...
1574,24,Washington Avenue Elementary School,The School District Of The Chathams,None,9,-1,-1,PK-3 & Ungraded,349,17.0
1578,3,Viola L. Sickles Elementary School,Fair Haven School District,None,10,-1,-1,PK-3,401,10.0
1581,6,Southern Boulvard School,The School District Of The Chathams,None,9,-1,-1,PK-3 & Ungraded,468,15.0
1588,13,Oak Tree Elementary School,Monroe Township Board Of Education School Dist...,None,9,-1,-1,PK-3 & Ungraded,700,16.0


In [58]:
df_all.to_csv('great_schools_ranking.csv')

In [59]:
df_rating_10 = df_all[df_all.rating_scores==10]
df_rating_9 = df_all[df_all.rating_scores==9]
df_rating_8 = df_all[df_all.rating_scores==8]

In [61]:
rating_10_per_district = df_rating_10.groupby('district').count()[['school']]
rating_9_per_district = df_rating_9.groupby('district').count()[['school']]
rating_8_per_district = df_rating_8.groupby('district').count()[['school']]

In [67]:
rating_10_per_district.columns = ['10']
rating_9_per_district.columns = ['9']
rating_8_per_district.columns = ['8']

In [68]:
count_per_district = rating_10_per_district.merge(rating_9_per_district, how='outer', left_index=True, right_index=True)
count_per_district = count_per_district.merge(rating_8_per_district, how='outer', left_index=True, right_index=True)

In [70]:
count_per_district.fillna(0, inplace=True)
count_per_district['10'] = count_per_district['10'].astype(int)
count_per_district['9'] = count_per_district['9'].astype(int)
count_per_district['8'] = count_per_district['8'].astype(int)

In [71]:
count_per_district['weighted count'] = count_per_district['10'] * 100 + \
                                       count_per_district['9'] * 10 + \
                                       count_per_district['8'] * 1

In [72]:
count_per_district = count_per_district.sort_values('weighted count', ascending=False)

In [73]:
count_per_district

,10,9,8,weighted count
district,,,,
Millburn Township School District,6,0,0,600
Livingston Public School District,4,3,0,430
Ridgewood Public School District,4,2,0,420
Fair Lawn Public School District,4,0,1,401
Summit Public School District,3,2,0,320
...,...,...,...,...
Jackson Township School District,0,0,1,1
Lavallette Borough Board Of Education School District,0,0,1,1
Leonia Public School District,0,0,1,1


In [74]:
count_per_district.to_csv('count_per_district_great_schools.csv')

In [112]:
df_gs = count_per_district.reset_index()
df_niche = pd.read_csv('count_per_district.csv')

In [113]:
df_niche.head()

,district,A+,A,weighted count
0,Millburn Township School District,6,0,60
1,Ridgewood Public School District,5,1,51
2,Livingston Board of Education School District,4,2,42
3,Summit Public School District,3,2,32
4,Tenafly Public School District,3,1,31


In [114]:
df_gs.head()

,district,10,9,8,weighted count
0,Millburn Township School District,6,0,0,600
1,Livingston Public School District,4,3,0,430
2,Ridgewood Public School District,4,2,0,420
3,Fair Lawn Public School District,4,0,1,401
4,Summit Public School District,3,2,0,320


In [115]:
df_gs['district'] = df_gs.district.apply(lambda text : ' '.join(text.split()).title())
df_niche['district'] = df_niche.district.apply(lambda text : ' '.join(text.split()).title())

In [123]:
df_both = df_gs.merge(df_niche, how='outer', left_on='district', right_on='district')

In [124]:
df_both

,district,10,9,8,weighted count_x,A+,A,weighted count_y
0,Millburn Township School District,6.0,0.0,0.0,600.0,6.0,0.0,60.0
1,Livingston Public School District,4.0,3.0,0.0,430.0,NaN,NaN,NaN
2,Ridgewood Public School District,4.0,2.0,0.0,420.0,5.0,1.0,51.0
3,Fair Lawn Public School District,4.0,0.0,1.0,401.0,NaN,NaN,NaN
4,Summit Public School District,3.0,2.0,0.0,320.0,3.0,2.0,32.0
...,...,...,...,...,...,...,...,...
275,Monmouth Beach,NaN,NaN,NaN,NaN,0.0,1.0,1.0
276,Little Falls Township Public Schools,NaN,NaN,NaN,NaN,0.0,1.0,1.0
277,Lavallette,NaN,NaN,NaN,NaN,0.0,1.0,1.0
278,Kinnelon School District,NaN,NaN,NaN,NaN,0.0,1.0,1.0


In [121]:
df_both = df_both.fillna(0)
df_both['weighted count_x'] = df_both['weighted count_x'].astype(int)
df_both['weighted count_y'] = df_both['weighted count_y'].astype(int)

In [125]:
df_both.to_csv('combined_count.csv')